In [33]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import pandas as pd
from datetime import datetime

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second
THRESHOLD_X = 2000
THRESHOLD_Y = 0.6

In [34]:
# Creating dataframe for frequency
threshold_df = pd.DataFrame(columns=['Frequency', 'Loudness', 'TimeStamp'])

# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])
plt.axvline(x = THRESHOLD_X, color = 'r', label = 'axvline - full height')
plt.axhline(y = THRESHOLD_Y, color = 'r', label = 'axhline - full width')

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128

    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_int)
    temp = np.abs(yf[0:CHUNK]) / (128 * CHUNK)
    np.clip(temp, 0, 20000)
    line_fft.set_ydata(temp)
    (x_line_fft, y_line_fft) = line_fft.get_data()
    x_line_fft = np.array(x_line_fft)[10:]
    y_line_fft = np.array(y_line_fft)[10:]
    threshold_arr = np.where(np.logical_and(x_line_fft > THRESHOLD_X, y_line_fft > THRESHOLD_Y))
    if (len(threshold_arr[0]) > 0):
        threshold_df.append({'Frequency': x_line_fft[threshold_arr[0][0]], 'Loudness': y_line_fft[threshold_arr[0][0]], 
        'TimeStamp': datetime.now().strftime("%d/%m/%Y %H:%M:%S")}, ignore_index=True)

    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    except TclError:
        print('stream stopped')
        break

display(threshold_df)
threshold_df.to_csv('output.csv')

stream started
stream stopped


,Frequency,Loudness,TimeStamp
